<a href="https://colab.research.google.com/github/xbyork/EscapeRoom/blob/main/EDUC283_Behaviorism_EAS_Mock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧩 Escape Room Generator (OpenAI + CSV RAG) — No UI + Auto GitHub Push

This Colab notebook builds a **single self-contained HTML** escape-room game from your **CSV standards** using **OpenAI** with a minimal RAG step, **then automatically pushes the generated HTML** to your GitHub repo:

- **Repo:** `xbyork/EscapeRoom`
- **Branch:** `main`
- **Folder:** `games/`

> **Token**: The notebook will look for your GitHub Personal Access Token (PAT) under the key **`EscapeRoomFineTunedToken`** via Colab `userdata`, environment, or a secure input prompt.


In [ ]:
!pip -q install --upgrade openai pandas python-dotenv
import os, re, glob, shutil, pathlib, textwrap

import pandas as pd
from dotenv import load_dotenv
load_dotenv()  # If you uploaded a .env with OPENAI_API_KEY

from google.colab import userdata
import os

openai_key = userdata.get('EscapeRAG')
os.environ['EscapeRAG'] = openai_key

# === Configuration (edit as needed) ===
GRADE       = 'high'
TOPICS      = 'phishing,passwords'
PUZZLES     = 5
DIFFICULTY  = 'difficult'
SUBJECT     = 'cybersecurity'
MODEL_NAME  = 'gpt-5-chat-latest'
SAVE_TO_DRIVE = True  # set False to skip Drive copy
DRIVE_DIR = '/content/drive/MyDrive/Colab Notebooks'  # where to copy in Drive

# API key: from env var FIRST; if not present, prompt once.
OPENAI_API_KEY = os.getenv('EscapeRAG')
if not OPENAI_API_KEY:
    try:
        OPENAI_API_KEY = input('EscapeRAG').strip()
    except EOFError:
        OPENAI_API_KEY = ''
if not OPENAI_API_KEY:
    raise ValueError('Missing OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)
print('✅ OpenAI client ready.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 58.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
✅ OpenAI client ready.


In [ ]:
# === Load standards.csv (from /content or upload) ===
from google.colab import files
import os

csv_path = '/content/standards.csv'
if not os.path.isfile(csv_path):
    print('Upload your standards CSV (e.g., standards.csv) ...')
    uploaded = files.upload()
    if not uploaded:
        raise ValueError('No CSV uploaded.')
    csv_path = list(uploaded.keys())[0]

df = pd.read_csv(csv_path)
print('📄 Loaded:', csv_path, '| Rows:', len(df))
display(df.head(8))


Upload your standards CSV (e.g., standards.csv) ...


Saving standards.csv to standards.csv
📄 Loaded: standards.csv | Rows: 10


,standard_id,grade_band,topic,standard_text
0,CSDF-1.1,elementary,phishing,Recognize when an online message looks suspici...
1,CSDF-1.2,elementary,passwords,Explain why you should not share your password...
2,CSDF-2.1,middle,phishing,Identify suspicious emails and messages that m...
3,CSDF-2.2,middle,passwords,"Demonstrate creation of strong, unique passwords"
4,CSDF-2.3,middle,networks,Describe what a firewall does to protect devic...
5,NICE-K12-PRIV-1,high,privacy,Explain the importance of protecting personall...
6,NICE-K12-NET-2,high,networks,Compare how different types of network defense...
7,NICE-K12-PHISH-3,high,phishing,Analyze real-world phishing examples and expla...


In [ ]:
# === Core generation (no UI; selections are hard-coded above) ===
def tiny_retrieve(df: pd.DataFrame, grade_band: str, topics: str):
    topics_list = [t.strip().lower() for t in topics.split(',') if t.strip()]
    mask = (df['grade_band'].str.lower() == grade_band.lower()) & (df['topic'].str.lower().isin(topics_list))
    return df[mask].copy(), topics_list

SYSTEM_PROMPT = '''\
You are the Escape Room Game Master & Curriculum Aligner. Use retrieved standards (CSV lines provided) to design K–12 cybersecurity escape-room puzzles.

How You Work:
1) Parse teacher input (grade_band, subject, topic(s), difficulty, #puzzles).
2) Retrieve (RAG): use ONLY the provided rows. Show standards text to players.
3) Plan a short narrative + list of puzzle objectives mapped to the retrieved standards.
4) Generate puzzles (MCQ/SHORT/MATCH) with: question, choices (if MCQ), correct answer(s), explanation (why it matters), and 3 hints.
5) Present sequentially in an escape-room storyline (locks/doors/codes) with feedback: **Correct!/Incorrect!**, brief reason, game effect, and HINT/SCORE/SKIP controls.
6) Safety: no hacking/malware/exploit instructions; keep age-appropriate; school/home/device contexts.

Output Requirements:
- Produce EXACTLY ONE self-contained HTML file (inline CSS/JS, no external CDNs).
- Include narrative intro; sequential puzzles; hint/skip/score; lock-opening feedback.
- Include a persistent “Download This Game (.html)” button that downloads the entire current page using Blob + outerHTML.
- Set window.__gameGrade and window.__gameTopics for filename.

Do not output any extra commentary—HTML only.
'''

def build_user_message(grade, subject, topics, puzzles, difficulty, retrieved_df):
    teacher_input = f"grade={grade}; subject={subject}; topic={topics}; puzzles={puzzles}; difficulty={difficulty}"
    retrieved_block = "\n".join(
        f"{r.standard_id}\t{r.grade_band}\t{r.topic}\t{r.standard_text}"
        for r in retrieved_df.itertuples(index=False)
    )
    return f'''\
Teacher input:
{teacher_input}

Use ONLY these retrieved rows to ground the puzzles:
RETRIEVED_STANDARDS_START
{retrieved_block}
RETRIEVED_STANDARDS_END

Output: Exactly ONE complete self-contained HTML/JS file with a built-in "Download This Game (.html)" button.
'''

def call_openai_html(api_key, system_prompt, user_message, model=MODEL_NAME):
    _client = OpenAI(api_key=api_key)
    resp = _client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_message}
        ]
    )
    html = getattr(resp, 'output_text', None)
    if not html:
        # Fallback for older response shapes
        try:
            html = resp.choices[0].message.content
        except Exception as e:
            raise RuntimeError(f'Could not read response text: {e}')
    return html

# RAG step
retrieved_df, topics_list = tiny_retrieve(df, GRADE, TOPICS)
if retrieved_df.empty:
    raise ValueError('No matching standards found. Adjust GRADE/TOPICS to match your CSV.')

# Build messages and call model
user_msg = build_user_message(GRADE, SUBJECT, TOPICS, PUZZLES, DIFFICULTY, retrieved_df)
print('⏳ Generating HTML...')
html = call_openai_html(OPENAI_API_KEY, SYSTEM_PROMPT, user_msg)

# Save to /content
import time
safe_topics = re.sub(r'[^A-Za-z0-9]+', '_', TOPICS).strip('_').lower() or 'cyber'

# ✅ Add timestamp to filename (YYYYMMDD-HHMMSS)
timestamp = time.strftime("%Y%m%d-%H%M%S")
out_name = f"cyber_escape_{GRADE}_{safe_topics}_{timestamp}.html"

out_path = f"/content/{out_name}"
with open(out_path, 'w', encoding='utf-8') as f:
    f.write(html)
print('💾 Saved:', out_path)

# Export a global so other cells/scripts can find the latest artifact
globals()['LAST_HTML_PATH'] = out_path
print('LAST_HTML_PATH =', LAST_HTML_PATH)

# Optional: copy to Google Drive
if SAVE_TO_DRIVE:
    try:
        from google.colab import drive
        if not os.path.ismount('/content/drive'):
            drive.mount('/content/drive')
        pathlib.Path(DRIVE_DIR).mkdir(parents=True, exist_ok=True)
        dest = os.path.join(DRIVE_DIR, os.path.basename(out_path))
        shutil.copy2(out_path, dest)
        print('✅ Also saved to Google Drive:', dest)
    except Exception as e:
        print('⚠️ Drive copy skipped/failed:', e)


⏳ Generating HTML...
💾 Saved: /content/cyber_escape_high_phishing_passwords_20251103-160421.html
LAST_HTML_PATH = /content/cyber_escape_high_phishing_passwords_20251103-160421.html
Mounted at /content/drive
✅ Also saved to Google Drive: /content/drive/MyDrive/Colab Notebooks/cyber_escape_high_phishing_passwords_20251103-160421.html


In [ ]:
# === GitHub push configuration ===
GITHUB_REPO      = 'xbyork/EscapeRoom'   # owner/repo
GITHUB_BRANCH    = 'main'
GITHUB_SUBDIR    = ''                    # '' = push to repo root; e.g., 'games' for subfolder

# === Secrets retrieval (uses the exact key name you provided) ===
from google.colab import userdata
import os

OPENAI_API_KEY = os.getenv('EscapeRAG') or userdata.get('EscapeRAG') or ''
if not OPENAI_API_KEY:
    OPENAI_API_KEY = input('Enter your OPENAI API key (EscapeRAG): ').strip()

# ✅ Fine-grained token name as provided
GITHUB_PAT = userdata.get('EscapeRoomFineTunedToken') or os.getenv('EscapeRoomFineTunedToken') or ''
if not GITHUB_PAT:
    GITHUB_PAT = input('Enter your GitHub PAT (EscapeRoomFineTunedToken): ').strip()

if not GITHUB_PAT:
    raise ValueError('Missing GitHub PAT (EscapeRoomFineTunedToken).')


# === Push the generated HTML to GitHub: xbyork/EscapeRoom (main) in games/ ===
import subprocess, shutil, os

def run(cmd, cwd=None, check=True):
    print("•", " ".join(cmd))
    res = subprocess.run(cmd, cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if res.stdout:
        print(res.stdout)
    if check and res.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}")
    return res

def git_push_html(html_path, repo=GITHUB_REPO, branch=GITHUB_BRANCH, subdir=GITHUB_SUBDIR, pat=GITHUB_PAT):
    import os # Moved import here to ensure 'os' is available

    if not os.path.isfile(html_path):
        raise FileNotFoundError(f"HTML file not found: {html_path}")

    import base64, json, requests, urllib.parse, time

    owner, name = repo.split('/', 1)
    api_base = f'https://api.github.com/repos/{owner}/{name}'
    headers = {
        'Authorization': f'Bearer {pat}',
        'Accept': 'application/vnd.github+json'
    }

    filename = os.path.basename(html_path)
    if subdir:
        path_in_repo = f'{subdir.strip("/")}/{filename}'
    else:
        path_in_repo = filename

    with open(html_path, 'rb') as f:
        content_b64 = base64.b64encode(f.read()).decode('utf-8')

    # Check if file exists to decide between create vs update
    get_url = f'{api_base}/contents/{urllib.parse.quote(path_in_repo)}?ref={branch}'
    r = requests.get(get_url, headers=headers)
    sha = r.json().get('sha') if r.status_code == 200 else None

    commit_msg = f'Add/Update {filename} (Colab auto-publish)'
    put_url = f'{api_base}/contents/{urllib.parse.quote(path_in_repo)}'
    payload = {
        'message': commit_msg,
        'content': content_b64,
        'branch': branch
    }
    if sha:
        payload['sha'] = sha

    resp = requests.put(put_url, headers=headers, data=json.dumps(payload))
    if resp.status_code not in (200, 201):
        raise RuntimeError(f'GitHub API error: {resp.status_code} {resp.text}')

    print(f'✅ Pushed to GitHub: https://github.com/{repo}/blob/{branch}/{path_in_repo}')
    owner, name = repo.split('/', 1)
    print(f'✅ Page in GitHub Pages (if Pages enabled): https://{owner}.github.io/{name}/{path_in_repo}')

# Ensure the token is loaded in this runtime before pushing (belt & suspenders)
GITHUB_PAT = userdata.get('EscapeRoomFineTunedToken') or os.getenv('EscapeRoomFineTunedToken') or GITHUB_PAT

git_push_html(LAST_HTML_PATH)

✅ Pushed to GitHub: https://github.com/xbyork/EscapeRoom/blob/main/cyber_escape_high_phishing_passwords_20251103-160421.html
✅ Page in GitHub Pages (if Pages enabled): https://xbyork.github.io/EscapeRoom/cyber_escape_high_phishing_passwords_20251103-160421.html


In [ ]:
# === Build & Push index.html that lists ALL escape-room games (newest first) ===
# - Stores timestamps in UTC (ISO) and converts to the viewer's local timezone in the browser
# - Assumes your repo is xbyork/EscapeRoom with GitHub Pages enabled from main (root)
# - Requires your fine-grained PAT saved in Colab "Secrets" as EscapeRoomFineTunedToken

import os, json, base64, requests, urllib.parse
from datetime import datetime, timezone
from google.colab import userdata

# --- Repo config ---
GITHUB_REPO   = "xbyork/EscapeRoom"   # owner/repo
GITHUB_BRANCH = "main"
GITHUB_SUBDIR = ""                    # '' for repo root; else e.g. 'games'

# --- Auth (PAT) ---
GITHUB_PAT = userdata.get("EscapeRoomFineTunedToken") or os.getenv("EscapeRoomFineTunedToken") or ""
if not GITHUB_PAT:
    GITHUB_PAT = input("Enter your GitHub PAT (EscapeRoomFineTunedToken): ").strip()
if not GITHUB_PAT:
    raise ValueError("Missing GitHub PAT (EscapeRoomFineTunedToken).")

owner, repo_name = GITHUB_REPO.split("/", 1)
API_BASE = f"https://api.github.com/repos/{owner}/{repo_name}"
HEADERS = {
    "Authorization": f"Bearer {GITHUB_PAT}",
    "Accept": "application/vnd.github+json",
}

def list_repo_html_files(branch=GITHUB_BRANCH, subdir=GITHUB_SUBDIR):
    """
    Returns list of (path, name) for all .html files in subdir (or root if blank), excluding index.html.
    """
    path = subdir.strip("/")
    url = f"{API_BASE}/contents/{urllib.parse.quote(path)}" if path else f"{API_BASE}/contents"
    resp = requests.get(url, headers=HEADERS, params={"ref": branch})
    if resp.status_code != 200:
        raise RuntimeError(f"List contents error: {resp.status_code} {resp.text}")

    html_files = []
    for item in resp.json():
        if item.get("type") == "file" and item.get("name", "").lower().endswith(".html"):
            if item["name"].lower() == "index.html":
                continue
            html_files.append((item["path"], item["name"]))
    return html_files

def latest_commit_iso_for_path(path, branch=GITHUB_BRANCH):
    """
    Queries the latest commit touching `path` and returns its ISO UTC datetime string.
    """
    commits_url = f"{API_BASE}/commits"
    r = requests.get(commits_url, headers=HEADERS, params={"path": path, "sha": branch, "per_page": 1})
    if r.status_code != 200:
        # Fallback to now if we can't fetch commit metadata
        return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
    data = r.json()
    if not data:
        return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
    # Prefer the commit author date
    try:
        iso = data[0]["commit"]["author"]["date"]
        # Ensure trailing Z
        if iso.endswith("Z"):
            return iso
        # Normalize to Z if not
        return datetime.fromisoformat(iso.replace("Z", "+00:00")).astimezone(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
    except Exception:
        return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

def get_existing_sha(path_in_repo, branch=GITHUB_BRANCH):
    url = f"{API_BASE}/contents/{urllib.parse.quote(path_in_repo)}"
    r = requests.get(url, headers=HEADERS, params={"ref": branch})
    if r.status_code == 200:
        return r.json().get("sha")
    return None

def push_file(path_in_repo, content_text, commit_msg, branch=GITHUB_BRANCH):
    """
    Creates or updates a file via Contents API.
    """
    sha = get_existing_sha(path_in_repo, branch)
    put_url = f"{API_BASE}/contents/{urllib.parse.quote(path_in_repo)}"
    payload = {
        "message": commit_msg,
        "content": base64.b64encode(content_text.encode("utf-8")).decode("utf-8"),
        "branch": branch
    }
    if sha:
        payload["sha"] = sha
    r = requests.put(put_url, headers=HEADERS, data=json.dumps(payload))
    if r.status_code not in (200, 201):
        raise RuntimeError(f"GitHub API error pushing {path_in_repo}: {r.status_code} {r.text}")
    return r.json()

def build_index_html(rows):
    """
    rows: list of dicts with keys: name (filename), href (link), utc (ISO UTC string)
    Creates HTML that:
      - stores UTC in data-utc
      - converts to viewer local time with JS
      - sorts already applied (latest first)
    """
    # Basic styling + client timezone conversion
    html = f"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<title>Escape Room Games</title>
<style>
  body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial, sans-serif; margin: 40px; }}
  h1 {{ margin-top: 0; text-align: left; }}   /* ensure title is also left aligned */
  table {{ border-collapse: collapse; width: 100%; max-width: 1000px; }}
  th, td {{ border: 1px solid #ddd; padding: 10px; }}
  th {{
    background: #f6f6f6;
    text-align: left;   /* <-- Key fix */
    font-weight: 600;
  }}
  tr:hover {{ background: #fafafa; }}
  .meta {{ color: #666; font-size: 0.9em; margin-bottom: 16px; text-align: left; }}
</style>
</head>
<body>
  <h1>Escape Room Games</h1>
  <div class="meta">This list shows times in your local timezone. Source of truth stored in UTC.</div>
  <table>
    <thead>
      <tr>
        <th style="width:60%"  >File</th>
        <th style="width:40%"  >Date</th>
      </tr>
    </thead>
    <tbody>
      {"".join(
        f"<tr><td><a href='{r['href']}' target='_blank' rel='noopener'>{r['name']}</a></td>"
        f"<td data-utc='{r['utc']}'></td></tr>"
        for r in rows
      )}
    </tbody>
  </table>

  <script>
    // Convert all UTC timestamps to the viewer's local timezone
    const opts = {{
      year: "numeric", month: "short", day: "numeric",
      hour: "2-digit", minute: "2-digit", second: "2-digit",
      timeZoneName: "short"
    }};
    document.querySelectorAll("td[data-utc]").forEach(td => {{
      const iso = td.getAttribute("data-utc");
      const d = new Date(iso); // parse UTC
      td.textContent = d.toLocaleString(undefined, opts);
    }});
  </script>
</body>
</html>"""
    return html

# --- Build list from repo, get last commit timestamps, sort newest first ---
html_files = list_repo_html_files()
entries = []
for path, name in html_files:
    iso_utc = latest_commit_iso_for_path(path)
    # href relative to repo root (Pages from root)
    href = name if not GITHUB_SUBDIR else f"{GITHUB_SUBDIR.strip('/')}/{name}"
    entries.append({"name": name, "href": href, "utc": iso_utc})

# Sort by UTC desc (latest first)
entries.sort(key=lambda r: r["utc"], reverse=True)

# Build index.html
index_html = build_index_html(entries)

# Path in repo
index_path_in_repo = "index.html" if not GITHUB_SUBDIR else f"{GITHUB_SUBDIR.strip('/')}/index.html"

# Push to GitHub
push_file(index_path_in_repo, index_html, "Rebuild index with left-aligned headers")
# Print links
pages_url = f"https://{owner}.github.io/{repo_name}/" if not GITHUB_SUBDIR else f"https://{owner}.github.io/{repo_name}/{GITHUB_SUBDIR.strip('/')}/"
blob_url  = f"https://github.com/{GITHUB_REPO}/blob/{GITHUB_BRANCH}/{index_path_in_repo}"

print(f"✅ Published index: {pages_url}")
print(f"🔗 GitHub (blob view): {blob_url}")

✅ Published index: https://xbyork.github.io/EscapeRoom/
🔗 GitHub (blob view): https://github.com/xbyork/EscapeRoom/blob/main/index.html
